# Download Planet Basemap Scenes for ARTS Using Orders API
This script downloads the 2018 July-Sept. basemap grids which contain delineated RTS.

In [1]:
import os
import json
import requests
import urllib.request
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pprint import pprint
import math
import time
import ast

In [2]:
# Get Planet API Key
%load_ext dotenv
%dotenv

api_key = os.getenv('PL_BM_API_KEY')

## Define Functions

In [3]:
def makemydir(dir_path):
    try:
        os.makedirs(dir_path)
    except OSError:
        pass

In [4]:
def recu_down(url, filename): # recurrent download with ContentTooShortError
    try:
        urllib.request.urlretrieve(url,filename)
    except urllib.error.ContentTooShortError:
        print('Download failed. Trying again...')
        recu_down(url, filename)

# Import Data

In [5]:
grids_filtered = gpd.read_file('../data/planet_grids_for_arts_closest_year.geojson')

In [6]:
grids_filtered

,planet_basemap_year,id,link,replicate,geometry
0,2017.0,0-1515,https://api.planet.com/basemaps/v1/mosaics/a2b...,2,"POLYGON ((-1950841.755 1938908.115, -1944872.9..."
1,2017.0,0-1516,https://api.planet.com/basemaps/v1/mosaics/a2b...,1,"POLYGON ((-1944872.974 1932975.846, -1938922.4..."
2,2017.0,0-1549,https://api.planet.com/basemaps/v1/mosaics/a2b...,1,"POLYGON ((-1757802.562 1747049.776, -1752423.2..."
3,2017.0,0-1553,https://api.planet.com/basemaps/v1/mosaics/a2b...,1,"POLYGON ((-1736383.836 1725762.072, -1731069.9..."
4,2022.0,1-1532,https://api.planet.com/basemaps/v1/mosaics/c21...,10,"POLYGON ((-1857451.331 1834795.702, -1851767.6..."
...,...,...,...,...,...
42800,2016.0,998-3050,https://api.planet.com/basemaps/v1/mosaics/e95...,1,"POLYGON ((-1957119.390 -1812485.926, -1954122...."
42801,2016.0,999-2774,https://api.planet.com/basemaps/v1/mosaics/e95...,2,"POLYGON ((-2982359.184 -2770469.941, -2977798...."
42802,2016.0,999-2775,https://api.planet.com/basemaps/v1/mosaics/e95...,1,"POLYGON ((-2977798.589 -2766233.364, -2973244...."
42803,2016.0,999-3012,https://api.planet.com/basemaps/v1/mosaics/e95...,1,"POLYGON ((-2071498.442 -1924323.601, -2068327...."


In [7]:
print(grids_filtered.planet_basemap_year.sort_values().unique())
grids_filtered_annual = [
    grids_filtered[grids_filtered.planet_basemap_year == year] 
    for year 
    in grids_filtered.planet_basemap_year.sort_values().unique()
]
print([len(df.index) for df in grids_filtered_annual])

grids_filtered_annual_5000 = [
    np.array_split(annual_grids, math.ceil(len(annual_grids)/5000)) 
                   for annual_grids 
                   in grids_filtered_annual
                   ]
pprint([[len(df.index) for df in year] for year in grids_filtered_annual_5000])

[2016. 2017. 2018. 2019. 2020. 2021. 2022. 2023.]
[20720, 5079, 4175, 4331, 3949, 3962, 337, 252]
[[4144, 4144, 4144, 4144, 4144],
 [2540, 2539],
 [4175],
 [4331],
 [3949],
 [3962],
 [337],
 [252]]


In [ ]:
makemydir('../data/download/')
order_info_path = '../data/download/planet_basemap_orders.csv'
order_download_path = '../data/downloads/planet_basemap_downloads.csv'


for annual_grids in grids_filtered_annual_5000[5][0:1]:
    
    basemap_name = 'global_quarterly_' + str(int(annual_grids.planet_basemap_year.iloc[0])) + 'q3_mosaic'

    requested_list = []

    # split image info into list of chunks of approximately 5 basemaps to download
    n = 5
    grids_filtered_list = np.array_split(annual_grids, math.ceil(len(annual_grids)/n))
    
    count = 0
    
    # submit order for each item in the list
    for chunk in grids_filtered_list:
        count += 1
        
        # Try this section
        if os.path.exists(order_info_path):
            prior_orders = pd.read_csv(order_info_path,
                                       converters = {'order_info': ast.literal_eval})
            prior_orders = [row['order_info'] for idx, row in prior_orders.iterrows()]
            prior_names = [item['name'] for item in prior_orders]

        else:
            prior_names = []

    # Get chunk info
        item_ids = list(chunk.id)

        order_name = (
            basemap_name
            + '_'
            + item_ids[0]
            + '_'
            + item_ids[len(item_ids)-1]
        )

        print("-------------------------------------")
        print(count, ': ', order_name)

        if len(item_ids) > 0: # if images to download

            if order_name not in prior_names: # check if an order has already been placed

                print(len(item_ids), "items to be ordered for", order_name)

                # create the order info
                order_info = {
                    "name": order_name,
                    "source_type": "basemaps",
                    "products": [{
                        "mosaic_name": basemap_name,
                        "quad_ids": item_ids
                    }]
                }


                # Place order
                print("Requesting items...")
                request = requests.post('https://api.planet.com/compute/ops/orders/v2', 
                                        auth=(api_key, ''),
                                        json=order_info)
                if str(request) != '<Response [200]>':
                    time.sleep(5)
                    request = requests.post('https://api.planet.com/compute/ops/orders/v2', 
                                            auth=(api_key, ''),
                                            json=order_info)

                # wait while the order is queued and runs
                order_status = request.json()
                while order_status['state'] == 'queued':
                    time.sleep(1)
                    try:
                        order_status = requests.get(request.json()['_links']['_self'], 
                                                    auth=(api_key, '')).json()
                    except:
                        time.sleep(5)
                        order_status = requests.get(request.json()['_links']['_self'], 
                                                    auth=(api_key, '')).json()

                while order_status['state'] == 'running':
                    time.sleep(1)
                    try:
                        order_status = requests.get(request.json()['_links']['_self'], 
                                                    auth=(api_key, '')).json()
                    except:
                        time.sleep(5)
                        order_status = requests.get(request.json()['_links']['_self'], 
                                                    auth=(api_key, '')).json()

                # If the order succeeded, create and append order info into file
                if order_status['state'] == 'success':

                    print('Order succeeded.')
                    print("-------------------------------------")
                    print("\n")

                    requested_list.append([order_info, request])

                    order_df = pd.DataFrame({
                        'order_name': order_name,
                        'order_info': [request.json()]
                    })

                    order_df.to_csv(order_info_path,
                                    index = False,
                                    mode = 'a',
                                    header = not os.path.exists(order_info_path))

                # If the order failed, stop the code from running any farther
                elif order_status['state'] == 'failed':
                    # Place order
                    print("Trying order again...")
                    try:
                        request = requests.post('https://api.planet.com/compute/ops/orders/v2', 
                                                auth=(api_key, ''),
                                                json=order_info)
                    except:
                        time.sleep(5)
                        request = requests.post('https://api.planet.com/compute/ops/orders/v2', 
                                                auth=(api_key, ''),
                                                json=order_info)

                    # wait while the order is queued and runs
                    order_status = request.json()
                    while order_status['state'] == 'queued':
                        time.sleep(1)
                        try:
                            order_status = requests.get(request.json()['_links']['_self'], 
                                                        auth=(api_key, '')).json()
                        except:
                            time.sleep(5)
                            order_status = requests.get(request.json()['_links']['_self'], 
                                                        auth=(api_key, '')).json()

                    while order_status['state'] == 'running':
                        time.sleep(1)
                        try:
                            order_status = requests.get(request.json()['_links']['_self'], 
                                                        auth=(api_key, '')).json()
                        except:
                            time.sleep(5)
                            order_status = requests.get(request.json()['_links']['_self'], 
                                                        auth=(api_key, '')).json()

                    # If the order succeeded, create and append order info into file
                    if order_status['state'] == 'success':

                        print('Order succeeded.')
                        print("-------------------------------------")
                        print("\n")

                        requested_list.append([order_info, request])

                        order_df = pd.DataFrame({
                            'order_name': order_name,
                            'order_info': [request.json()]
                        })

                        order_df.to_csv(order_info_path,
                                        index = False,
                                        mode = 'a',
                                        header = not os.path.exists(order_info_path))

                    # If the order failed, stop the code from running any farther
                    elif order_status['state'] == 'failed':
                        print('last_message:', order_status['last_message'], '\nerror_hints:', order_status['error_hints'])
                        break

            else:
                print('Order already placed.')
                print("-------------------------------------")
                print("\n")


-------------------------------------
1 :  global_quarterly_2021q3_mosaic_1298-1577_132-1553
5 items to be ordered for global_quarterly_2021q3_mosaic_1298-1577_132-1553
Requesting items...
Order succeeded.
-------------------------------------


-------------------------------------
2 :  global_quarterly_2021q3_mosaic_132-1554_1328-1659
5 items to be ordered for global_quarterly_2021q3_mosaic_132-1554_1328-1659
Requesting items...
Order succeeded.
-------------------------------------


-------------------------------------
3 :  global_quarterly_2021q3_mosaic_1329-1658_133-1553
5 items to be ordered for global_quarterly_2021q3_mosaic_1329-1658_133-1553
Requesting items...
Order succeeded.
-------------------------------------


-------------------------------------
4 :  global_quarterly_2021q3_mosaic_133-1554_1330-1658
5 items to be ordered for global_quarterly_2021q3_mosaic_133-1554_1330-1658
Requesting items...
Order succeeded.
-------------------------------------


----------------

In [ ]:
# # In case I need to cancel orders quickly
# requests.post('https://api.planet.com/compute/ops/bulk/orders/v2/cancel',
#               auth=(api_key, ''))

In [9]:
request.json()

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/fce05e17-0eb4-474b-85da-641102c52619'},
 'created_on': '2024-09-30T18:29:19.238Z',
 'error_hints': [],
 'id': 'fce05e17-0eb4-474b-85da-641102c52619',
 'last_message': 'Preparing order',
 'last_modified': '2024-09-30T18:29:19.238Z',
 'name': 'global_quarterly_2020q3_mosaic_1387-1568_1387-1577',
 'products': [{'mosaic_name': 'global_quarterly_2020q3_mosaic',
   'quad_ids': ['1387-1568',
    '1387-1570',
    '1387-1573',
    '1387-1574',
    '1387-1577']}],
 'source_type': 'basemaps',
 'state': 'queued'}

In [19]:
order_status = requests.get(request.json()['_links']['_self'], 
                            auth=(api_key, '')).json()

In [20]:
order_status['state']

'success'

In [21]:
if order_status['state'] == 'success':

    print('Order succeeded.')
    print("-------------------------------------")
    print("\n")

    requested_list.append([order_info, request])

    order_df = pd.DataFrame({
        'order_name': order_name,
        'order_info': [request.json()]
    })

    order_df.to_csv(order_info_path,
                    index = False,
                    mode = 'a',
                    header = not os.path.exists(order_info_path))

Order succeeded.
-------------------------------------


